<a href="https://colab.research.google.com/github/doo52oh/Dacon/blob/main/%EC%9E%A1%EC%BC%80%EC%96%B4%EC%B6%94%EC%B2%9C/%EC%9E%A1%EC%BC%80%EC%96%B4%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_TabNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-tabnet==3.1.1

In [2]:
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from typing import Dict
from datetime import datetime
## preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import torch
from torch import nn
from pytorch_tabnet.tab_model  import TabNetClassifier 
from pytorch_tabnet.metrics import Metric

from sklearn.metrics import classification_report, f1_score, precision_score, recall_score



import warnings
warnings.filterwarnings(action='ignore')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DATA_PATH = "/content/drive/MyDrive/Dacon/JobCare/data/"
SUBMIT_PATH = "/content/drive/MyDrive/Dacon/JobCare/submit/"
SEED = 42

In [4]:
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')

d_code = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv', index_col=0).T.to_dict()
h_code = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv', index_col=0).T.to_dict()
l_code = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv', index_col=0).T.to_dict()

print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 35)
test_data.shape:  (46404, 34)


In [5]:
def add_code(
    df: pd.DataFrame,
    d_code: Dict[int, Dict[str, int]], 
    h_code: Dict[int, Dict[str, int]], 
    l_code: Dict[int, Dict[str, int]],
) -> pd.DataFrame:
    
    # Copy input data
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df

train_data = add_code(train_data, d_code, h_code, l_code)
test_data = add_code(test_data, d_code, h_code, l_code)
print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


In [6]:
def preprocessing_contents_open_dt(data):
    data['contents_open_dt'] = data['contents_open_dt'].astype('str')
    DATE = data['contents_open_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    DATE = pd.DataFrame(DATE)
    DATE = DATE.rename(columns = {'contents_open_dt': 'date'})
    
    DATE['Y'] = DATE['date'].apply(lambda x: x.timetuple()[0])
    DATE['M'] = DATE['date'].apply(lambda x: x.timetuple()[1])
    DATE['D'] = DATE['date'].apply(lambda x: x.timetuple()[2])
    DATE['id'] = data['id']
    
    data = data.merge(DATE, on = 'id', how = 'left')
    data = data.drop(columns = ['date', 'contents_open_dt'])
    return data

train_data = preprocessing_contents_open_dt(train_data)
test_data = preprocessing_contents_open_dt(test_data)

# 안전하게 확인하고 넘어 갑시다. 
train_data_labels = train_data['target']
train_data, test_data = train_data.align(test_data, join = 'inner', axis = 1)
train_data['target'] = train_data_labels
print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 65)
test_data.shape:  (46404, 64)


In [7]:
job_data = train_data.copy()
job_data_test = test_data.copy()

In [8]:
drop_cols = ['id', 'person_rn', 'contents_rn', 'Y', 'M', 'D','person_prefer_f','person_prefer_g']

job_df = job_data.drop(drop_cols, axis=1)
job_test_df = job_data_test.drop(drop_cols, axis=1)

In [9]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501951 entries, 0 to 501950
Data columns (total 57 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   h_m_match_yn            501951 non-null  bool 
 5   h_s_match_yn            501951 non-null  bool 
 6   person_attribute_a      501951 non-null  int64
 7   person_attribute_a_1    501951 non-null  int64
 8   person_attribute_b      501951 non-null  int64
 9   person_prefer_c         501951 non-null  int64
 10  person_prefer_d_1       501951 non-null  int64
 11  person_prefer_d_2       501951 non-null  int64
 12  person_prefer_d_3       501951 non-null  int64
 13  person_prefer_e         501951 non-null  int64
 14  person_prefer_h_1       501951 non-null  int64
 15  

In [10]:
np.random.seed(42)
if "Set" not in job_df.columns:
    job_df["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(job_df.shape[0],))

train_indices = job_df[job_df.Set=="train"].index
valid_indices = job_df[job_df.Set=="valid"].index


In [11]:
train = job_df.loc[job_df.Set == 'train'].drop("Set", axis=1)
valid = job_df.loc[job_df.Set == 'valid'].drop("Set", axis=1)
test = job_test_df

In [12]:
train.shape, valid.shape, test.shape

((401574, 57), (100377, 57), (46404, 56))

In [13]:
cat_idxs = []
cat_dims = []
for idx, col in enumerate(train.columns):
  print(idx,col)
  if 'match' not in col and col!='target': 
        le = LabelEncoder()
        le.fit(train[col].values)
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        train[col] = train[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        valid[col] = valid[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        test[col] = test[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        cat_idxs.append(idx)
        cat_dims.append(len(le_dict)+1)

0 d_l_match_yn
1 d_m_match_yn
2 d_s_match_yn
3 h_l_match_yn
4 h_m_match_yn
5 h_s_match_yn
6 person_attribute_a
7 person_attribute_a_1
8 person_attribute_b
9 person_prefer_c
10 person_prefer_d_1
11 person_prefer_d_2
12 person_prefer_d_3
13 person_prefer_e
14 person_prefer_h_1
15 person_prefer_h_2
16 person_prefer_h_3
17 contents_attribute_i
18 contents_attribute_a
19 contents_attribute_j_1
20 contents_attribute_j
21 contents_attribute_c
22 contents_attribute_k
23 contents_attribute_l
24 contents_attribute_d
25 contents_attribute_m
26 contents_attribute_e
27 contents_attribute_h
28 person_prefer_d_1_n
29 person_prefer_d_1_s
30 person_prefer_d_1_m
31 person_prefer_d_1_l
32 person_prefer_d_2_n
33 person_prefer_d_2_s
34 person_prefer_d_2_m
35 person_prefer_d_2_l
36 person_prefer_d_3_n
37 person_prefer_d_3_s
38 person_prefer_d_3_m
39 person_prefer_d_3_l
40 contents_attribute_d_n
41 contents_attribute_d_s
42 contents_attribute_d_m
43 contents_attribute_d_l
44 person_prefer_h_1_l
45 person_pre

In [14]:
X_train = train.drop('target',axis=1).values
y_train = train['target'].values
X_val = valid.drop('target',axis=1).values
y_val = valid['target'].values
X_test = test.values
# eval_set = (X_val,y_val)

In [15]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

((401574, 56), (401574,), (100377, 56), (100377,), (46404, 56))

In [16]:
X_test

array([[True, False, False, ..., 164, 40, 6],
       [False, False, False, ..., 237, 60, 12],
       [True, False, False, ..., 273, 67, 16],
       ...,
       [True, True, False, ..., 109, 27, 2],
       [True, True, False, ..., 109, 27, 2],
       [True, True, False, ..., 109, 27, 2]], dtype=object)

In [17]:
X_train

array([[True, True, True, ..., 237, 60, 12],
       [False, False, False, ..., 234, 59, 12],
       [False, False, False, ..., 237, 60, 12],
       ...,
       [True, True, True, ..., 73, 19, 2],
       [True, False, False, ..., 100, 25, 2],
       [True, True, True, ..., 113, 27, 2]], dtype=object)

## TabNet

In [18]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [19]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=3,
                       optimizer_fn=torch.optim.AdamW, # Any optimizer works here
                       mask_type='entmax', # "sparsemax",
                      )

Device used : cuda


In [20]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['logloss','auc', 'f1'],
    max_epochs=100 , patience=2,
    batch_size=1024,
    virtual_batch_size=256,
    num_workers=1,
    drop_last=False,
) 

epoch 0  | loss: 0.68477 | train_logloss: 0.66981 | train_auc: 0.6189  | train_f1: 0.62004 | val_logloss: 0.67018 | val_auc: 0.61717 | val_f1: 0.61787 |  0:00:45s
epoch 1  | loss: 0.66666 | train_logloss: 0.65807 | train_auc: 0.6441  | train_f1: 0.6201  | val_logloss: 0.66098 | val_auc: 0.63784 | val_f1: 0.61564 |  0:01:32s
epoch 2  | loss: 0.65593 | train_logloss: 0.64619 | train_auc: 0.66558 | train_f1: 0.62613 | val_logloss: 0.6523  | val_auc: 0.65433 | val_f1: 0.61804 |  0:02:20s
epoch 3  | loss: 0.6468  | train_logloss: 0.63587 | train_auc: 0.68271 | train_f1: 0.65894 | val_logloss: 0.64663 | val_auc: 0.66552 | val_f1: 0.64478 |  0:03:07s
epoch 4  | loss: 0.64049 | train_logloss: 0.63229 | train_auc: 0.69044 | train_f1: 0.66251 | val_logloss: 0.64471 | val_auc: 0.66742 | val_f1: 0.64793 |  0:03:54s
epoch 5  | loss: 0.6354  | train_logloss: 0.62395 | train_auc: 0.70143 | train_f1: 0.66331 | val_logloss: 0.6428  | val_auc: 0.67027 | val_f1: 0.64097 |  0:04:40s
epoch 6  | loss: 0.630

In [21]:
preds = clf.predict(X_test)
# preds = (preds[:,1]>0.5)*1

TypeError: ignored